In [1]:
from dataset import tweetProccess
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk, re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.sentiment.vader import SentimentIntensityAnalyzer as si


'''gets the data'''
dataHandler = tweetProccess.FileTreat()
content = dataHandler.getTweets()

no_features = 2000


#Use to download nltk content
#nltk.download()

'''defining a function to remove numbers'''
def tokenize(text):
    tokenized = []
    for word in nltk.word_tokenize(text):
        if re.search('[a-zA-Z]', word):
            tokenized.append(word.lower())
            
    return tokenized

'''defining a function to tokenize and get only vebs and adjectivies'''
def tokenizeStructures(text):
    #gets the type of each word in the text
    tags = pos_tag(word_tokenize(text))
    
    interesting = []
    for tag in tags:
        #removes tweet links
        if re.search('t.co+', tag[0]):
            continue
        
        if (tag[1] == 'VERB' or tag[1] == 'JJ') and len(tag[0]) > 2:
            interesting.append(tag[0])
    
    return interesting

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    

#print(get_neg_matrix(content))

In [2]:
cmps = 5

lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
tf_vectorizer = CountVectorizer(tokenizer=tokenizeStructures, max_features=no_features, stop_words='english')

tf = tf_vectorizer.fit_transform(content)
tf_feature_names = tf_vectorizer.get_feature_names()
topics = lda.fit(tf,labels)

NameError: name 'labels' is not defined

In [ ]:
print_top_words(lda, tf_feature_names, 10)

In [ ]:
print(lda.components_)

In [ ]:
doc_topic = lda.transform(tf)

for n in range(doc_topic.shape[0]):
    topic_most_pr = doc_topic[n].argmax()
    print("doc: {} topic: {}\n".format(n,topic_most_pr))

In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity
#dist = 1 - cosine_similarity(tf)

#print(dist)